In [378]:
import pandas as pd
from assign_unique_ids_functions import *
import numpy as np

In [379]:
df = pd.read_csv('../input/awards.csv.gz')

In [380]:
df.head()

,Gender,Race,Appointed.Date,Award.Type,Incident.Start.Date,Current.Award.Status,Award.Request.Date,Incident.End.Date,Resignation.Date,Award.ID,...,Rank,Birth.Year,Last.Promotion.Date,Requester.Full.Name,Ceremony.Date,Tracking.NO,Last.Name,Suffix.Name,First.Name,Middle.Initial
0,MALE,WHITE,2005-09-26,UNIT MERITORIOUS PERFORMANCE AWARD,2010-01-01,DENIED,2012-11-02,2010-12-31,NaN,341867,...,9171-SERGEANT OF POLICE,1971,NaN,"PAVON JR, RICARDO",NaN,-999,AARON,NaN,JEFFERY,M
1,MALE,WHITE,2005-09-26,DEPARTMENT COMMENDATION,2014-06-27,DENIED,2014-06-29,2014-06-27,NaN,632370,...,9171-SERGEANT OF POLICE,1971,NaN,"HENKELS, ADAM",NaN,-999,AARON,NaN,JEFFERY,M
2,MALE,WHITE,2005-09-26,HONORABLE MENTION,2014-06-27,DENIED,2014-07-04,2014-06-27,NaN,632978,...,9171-SERGEANT OF POLICE,1971,NaN,"HENKELS, ADAM",NaN,-999,AARON,NaN,JEFFERY,M
3,MALE,WHITE,2005-09-26,HONORABLE MENTION,2006-05-11,FINAL,2006-06-07,2006-05-11,NaN,30113,...,9171-SERGEANT OF POLICE,1971,NaN,"STINITES, ALEXANDER",NaN,-999,AARON,NaN,JEFFERY,M
4,MALE,WHITE,2005-09-26,HONORABLE MENTION,2006-12-02,FINAL,2007-01-31,2006-12-02,NaN,53794,...,9171-SERGEANT OF POLICE,1971,NaN,"CRAWFORD, PATRICK",NaN,-999,AARON,NaN,JEFFERY,M


In [381]:
df.columns

Index(['Gender', 'Race', 'Appointed.Date', 'Award.Type', 'Incident.Start.Date',
       'Current.Award.Status', 'Award.Request.Date', 'Incident.End.Date',
       'Resignation.Date', 'Award.ID', 'Current.Star', 'Rank', 'Birth.Year',
       'Last.Promotion.Date', 'Requester.Full.Name', 'Ceremony.Date',
       'Tracking.NO', 'Last.Name', 'Suffix.Name', 'First.Name',
       'Middle.Initial'],
      dtype='object')

In [382]:
id_cols = ["First.Name", "Last.Name", "Appointed.Date", "Gender"]
compare_cols = ["Middle.Initial", "Resignation.Date", "Current.Star"]
uid = 'awards_ID'

In [383]:
dfu = df[id_cols + compare_cols].drop_duplicates()
print(dfu.shape)
dfu.head()

(26134, 7)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,JEFFERY,AARON,2005-09-26,MALE,M,NaN,1424
82,KARINA,AARON,2005-09-26,FEMALE,NaN,NaN,20373
176,SUSAN,AARUP,2009-01-01,FEMALE,J,2009-02-27,-999
177,ANTHONY,ABBATE,1994-12-05,MALE,G,2009-12-22,-999
178,CARMEL,ABBATE,1969-01-06,MALE,G,2003-08-15,-999


In [384]:
rd_df = remove_duplicates(dfu, id_cols).reset_index(drop=True)
print(rd_df.head())
rd_df.shape

  First.Name Last.Name Appointed.Date  Gender Middle.Initial Resignation.Date  \
0    JEFFERY     AARON     2005-09-26    MALE              M              NaN   
1     KARINA     AARON     2005-09-26  FEMALE            NaN              NaN   
2      SUSAN     AARUP     2009-01-01  FEMALE              J       2009-02-27   
3    ANTHONY    ABBATE     1994-12-05    MALE              G       2009-12-22   
4     CARMEL    ABBATE     1969-01-06    MALE              G       2003-08-15   

   Current.Star  
0          1424  
1         20373  
2          -999  
3          -999  
4          -999  


(26094, 7)

In [385]:
rd_df.insert(0, 'NID', rd_df.index + 1)

In [386]:
#out_df = df.merge(rd_df[[uid] + id_cols], on = id_cols, how='inner')
#out_df.head()

In [387]:
kd_df = keep_duplicates(dfu, id_cols)
print(kd_df.shape)
kd_df.head()

(40, 7)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
214155,ANN,HERR,2002-06-11,FEMALE,M,2010-06-16,-999
214162,ANN,HERR,2002-06-11,FEMALE,M,NaN,-999
245404,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,-999
245405,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,8280
2276,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673


In [405]:
def resolve_conflicts(df, id_cols, resolve_cols, uid, starting_uid, , temp_fillna = -9999):
    out_df = pd.DataFrame()
    df = df.reset_index(drop=True)
    group_df = df.fillna(temp_fillna).groupby(id_cols, as_index=False)
    for key, group in group_df:
        group = group.reset_index(drop=True)
        conflicts = 0
        for col in compare_cols:
            non_nan = group[col].dropna()
            non_nan = non_nan[(non_nan != -999) & (non_nan != temp_fillna)]
            if len(non_nan.unique()) > 1:
                conflicts +=1
        if conflicts == 0:
            group.insert(0, temp_id, max_temp_id + 1)
            max_temp_id += 1
        else:
            group.insert(0, temp_i, max_temp_id + group.index + 1)
            max_temp_id += max(group.index) + 1
        out_df = out_df.append(group)
    out_df[out_df == temp_fillna] = np.nan
    return out_df

In [406]:
rc = resolve_conflicts(kd_df, id_cols,compare_cols, max(rd_df['NID']))

In [407]:
rc

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26095,ANN,HERR,2002-06-11,FEMALE,M,2010-06-16,-999
1,26095,ANN,HERR,2002-06-11,FEMALE,M,NaN,-999
0,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,-999
1,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,8280
0,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
1,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999
0,26098,DAVID,RAMOS,1994-07-05,MALE,NaN,2009-10-21,-999
1,26098,DAVID,RAMOS,1994-07-05,MALE,R,NaN,8560
0,26099,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,2334
1,26099,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,-999


In [408]:
df2 = df.merge(rd_df.append(rc), on=id_cols + compare_cols, how='left')

In [409]:
assert (df2[df2['NID'].isnull()].shape[0] == 0), "Uh-oh, some rows do not have IDs"

In [410]:
rc[(rc['First.Name'] == 'DAVID') & (rc['Last.Name'] == 'ADCOCK') & (rc['Appointed.Date'] == '2002-09-30')]

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
1,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999


In [411]:
print(tdf.shape)
tdf[id_cols + compare_cols].drop_duplicates()

(0, 22)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star


In [395]:
rc

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26095,ANN,HERR,2002-06-11,FEMALE,M,2010-06-16,-999
1,26095,ANN,HERR,2002-06-11,FEMALE,M,NaN,-999
2,26097,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,-999
3,26097,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,8280
4,26099,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
5,26099,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999
6,26101,DAVID,RAMOS,1994-07-05,MALE,NaN,2009-10-21,-999
7,26101,DAVID,RAMOS,1994-07-05,MALE,R,NaN,8560
8,26103,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,2334
9,26103,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,-999


In [412]:
df2[df2['NID'].isnull()]

,Gender,Race,Appointed.Date,Award.Type,Incident.Start.Date,Current.Award.Status,Award.Request.Date,Incident.End.Date,Resignation.Date,Award.ID,...,Birth.Year,Last.Promotion.Date,Requester.Full.Name,Ceremony.Date,Tracking.NO,Last.Name,Suffix.Name,First.Name,Middle.Initial,NID


In [413]:
nids = sorted(df2['NID'].unique())

In [415]:
assert nids[-1] == len(nids), print ('hey! got the wrong nids')

In [416]:
len(nids)

26124

In [417]:
max(nids)

26124

In [418]:
nids[-1]

26124